In [ ]:
import pandas as pd
from pyoso import Client    
import os
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path

# Define the path to the .env file in the parent directory
parent_dir = Path(__file__).resolve().parent.parent
load_dotenv(dotenv_path=parent_dir / '.env')

OSO_API_KEY = os.environ['OSO_API_KEY']

client = Client(api_key=OSO_API_KEY)

In [ ]:
## Helper functions
def stringify(lst):
    return ','.join(f"'{item}'" for item in lst)



### all projects and funding received on S7 Onchain builders

In [ ]:


GC_MECHANISM_NAME = 'Grants Council'
RF_MECHANISM_NAME = 'Retro Funding'


recipients = client.to_pandas(f"""
 with all_grants_all_projects as (
 
    SELECT
        funding_date,
        TRIM(json_extract_scalar(metadata, '$.application_name'), '"') AS title,
        to_project_name,
        grant_pool_name,
        CAST(TRIM(json_extract_scalar(metadata,'$.token_amount'),'"') AS DOUBLE) AS op_amount,
        amount AS usd_amount,
        TRIM(json_extract_scalar(metadata,'$.application_url'),'"') AS url,
        CASE
            WHEN grant_pool_name LIKE '%season%' THEN CONCAT('Season ',substr(grant_pool_name,-1,1))
            WHEN grant_pool_name LIKE '%s7%' THEN 'Season 7'
            WHEN grant_pool_name='retropgf2' THEN 'Season 3'
            WHEN grant_pool_name='retropgf3' THEN 'Season 4'
            WHEN grant_pool_name='retrofunding4' THEN 'Season 5'
            WHEN grant_pool_name IN('retrofunding5','retrofunding6') THEN 'Season 6'
            ELSE 'Unknown'
        END AS season,
        CASE WHEN grant_pool_name LIKE '%retro%' THEN 'Retro Funding' ELSE 'Grants Council' END AS mechanism,
        CONCAT(
            CASE
                WHEN grant_pool_name LIKE '%season%' THEN CONCAT('Season ',substr(grant_pool_name,-1,1))
                WHEN grant_pool_name LIKE '%s7%' THEN 'Season 7'
                WHEN grant_pool_name='retropgf2' THEN 'Season 3'
                WHEN grant_pool_name='retropgf3' THEN 'Season 4'
                WHEN grant_pool_name='retrofunding4' THEN 'Season 5'
                WHEN grant_pool_name IN('retrofunding5','retrofunding6') THEN 'Season 6'
                ELSE 'Unknown'
            END,
            ' - ',
            CASE WHEN grant_pool_name LIKE '%retro%' THEN '{RF_MECHANISM_NAME}' ELSE '{GC_MECHANISM_NAME}' END,
            ' - ',
            TRIM(json_extract_scalar(metadata,'$.application_name'),'"' )
        ) AS application
    FROM stg_ossd__current_funding
    WHERE
        from_funder_name='optimism'
        AND amount>0
)

select * from all_grants_all_projects where season = 'Season 7' and mechanism = 'Retro Funding'
    """)

ocb =recipients[recipients.grant_pool_name.str.contains('onchain_builders')]['to_project_name'].unique()
recipients = recipients[recipients.to_project_name.isin(ocb)]

In [4]:
# recipients.to_csv('../data/recipients_and_grants.csv')
recipients = pd.read_csv('../data/recipients_and_grants.csv')

In [10]:
recipients

,Unnamed: 0,funding_date,title,to_project_name,grant_pool_name,op_amount,usd_amount,url,season,mechanism,application
0,0,2025-04-05 00:00:00.000,CreateX,createx,retrofunding_s7_onchain_builders,1367.00,1025.2500,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - CreateX
1,1,2025-04-05 00:00:00.000,WOOFi,woonetwork,retrofunding_s7_onchain_builders,7718.74,5789.0550,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - WOOFi
2,2,2025-04-05 00:00:00.000,Exa App,exactly,retrofunding_s7_onchain_builders,576.41,432.3075,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Exa App
3,3,2025-04-05 00:00:00.000,Velodrome Finance,velodrome,retrofunding_s7_onchain_builders,66666.67,50000.0025,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Velodrome Finance
4,4,2025-04-05 00:00:00.000,pods.media,pods-media,retrofunding_s7_onchain_builders,5757.66,4318.2450,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - pods.media
...,...,...,...,...,...,...,...,...,...,...,...
858,1235,2025-08-06 00:00:00.000,Stargate Finance,stargate-finance,retrofunding_s7_devtooling,1764.72,1323.5400,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Stargate Finance
860,1248,2025-08-06 00:00:00.000,Party.app + Party Protocol,party-dao,retrofunding_s7_devtooling,2386.46,1789.8450,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Party.app + Party P...
863,1263,2025-08-06 00:00:00.000,Ethereum Attestation Service (EAS),ethereum-attestation-service,retrofunding_s7_devtooling,6307.54,4730.6550,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Ethereum Attestatio...
864,1270,2025-08-06 00:00:00.000,Splits,0x-splits,retrofunding_s7_devtooling,5456.48,4092.3600,https://atlas.optimism.io/project/op_atlas_id ...,Season 7,Retro Funding,Season 7 - Retro Funding - Splits


In [ ]:

recipients = pd.read_csv('../data/recipients_and_grants.csv')

In [ ]:
CHAINS = [
    'ARENAZ','AUTOMATA','BASE','BOB','CYBER','FRAX','HAM','INK','KROMA',
    'LISK','LYRA','METAL','MINT','MODE','OPTIMISM','ORDERLY','POLYNOMIAL',
    'RACE','REDSTONE','SHAPE','SONEIUM','SWAN','SWELL','UNICHAIN',
    'WORLDCHAIN','XTERIO','ZORA'
]
SOURCE_NAMES  = ['Github', 'Optimism', 'Base',    'Mode', 'Worldchain', 'Soneium', 'Ink',     'Unichain', 'Zora', 'Others']
TVL_METRIC = 'DefiLlama TVL (USD)'
FEES_METRIC = 'Fees Paid (ETH)'
TX_METRIC = 'Transaction Count'

GC_MECHANISM_NAME = 'Grants Council'
RF_MECHANISM_NAME = 'Retro Funding'


START_DATE = '2022-01-01'
END_DATE   = '2025-08-31'
OSO_PROJECT_NAMES = list(recipients['to_project_name'].dropna().unique())
ONCHAIN_METRICS = {
    'defillama_tvl_daily': 'DefiLlama TVL (USD)',
    'gas_fees_daily': 'Fees Paid (ETH)',
    'transactions_daily': 'Transaction Count',
#    'contract_invocations_daily': 'Contract Invocations Count',
}
# CODE_METRICS = {
#     'GITHUB_active_developers_monthly': 'Monthly Active Developers',
#     'GITHUB_full_time_developers_monthly': 'Monthly Full-Time Developers',
# }
ONCHAIN_METRIC_NAMES= [f"{c}_{m}" for c in CHAINS for m in ONCHAIN_METRICS.keys()]
# CODE_METRIC_NAMES = list(CODE_METRICS.keys())
# CODE_METRIC_LABELS = list(CODE_METRICS.values())

METRICS = ONCHAIN_METRIC_NAMES   # + CODE_METRIC_NAMES

# if REFRESH:
df_metrics_raw = client.to_pandas(f"""
WITH metrics AS (
    SELECT DISTINCT
        sample_date,
        project_name,
        metric_name,
        amount,
        SUM(amount) OVER (
            PARTITION BY project_name, metric_name
            ORDER BY sample_date
            RANGE BETWEEN INTERVAL '29' DAY PRECEDING AND CURRENT ROW
        ) AS rolling_30d_amount
    FROM timeseries_metrics_by_project_v0
    JOIN metrics_v0 USING (metric_id)
    JOIN projects_v1 USING (project_id)
    WHERE
        project_name IN ({stringify(OSO_PROJECT_NAMES)})
        AND metric_name IN ({stringify(METRICS)})
        AND sample_date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
    ORDER BY sample_date, project_name, metric_name
)
SELECT
    sample_date,
    project_name,   
    metric_name,
    amount,
    CASE
    WHEN metric_name LIKE '%daily' THEN rolling_30d_amount /     30
    ELSE amount
    END AS rolling_30d_amount
FROM metrics
""")
df_metrics_raw['chain'] = df_metrics_raw['metric_name'].apply(lambda x: x.split('_')[0])
df_metrics_raw['chain'] = df_metrics_raw['chain'].apply(lambda x: x.title() if x.title() in SOURCE_NAMES else 'Others')

df_metrics_raw['metric'] = df_metrics_raw['metric_name'].apply(lambda x: '_'.join(x.split('_')[1:]))
# df_metrics_raw['metric'] = df_metrics_raw['metric'].apply(lambda x: ONCHAIN_METRICS.get(x, CODE_METRICS.get(f'GITHUB_{x}')))
df_metrics_raw['sample_date'] = pd.to_datetime(df_metrics_raw['sample_date'])
df_metrics_raw.drop(columns=['metric_name'], inplace=True)

df_metrics_raw = df_metrics_raw.groupby(['sample_date', 'project_name', 'chain', 'metric']).sum().reset_index()
df_metrics_raw.to_parquet("../data/ocb_metrics_raw.parquet")




## Eth prices

In [ ]:
import requests
import pandas as pd
coingecko_api_key= "" 
url = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=364&interval=daily"

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": ""
}

response = requests.get(url, headers=headers)

print(response.text)

{"prices":[[1725408000000,2437.1471557556497],[1725494400000,2448.30785784089],[1725580800000,2367.7700509506026],[1725667200000,2223.780428382685],[1725753600000,2272.6972849951912],[1725840000000,2296.4029520764498],[1725926400000,2359.565766831627],[1726012800000,2389.4740111256497],[1726099200000,2342.9639236941434],[1726185600000,2362.300238174588],[1726272000000,2445.08621032297],[1726358400000,2418.0828083179904],[1726444800000,2322.6433317480564],[1726531200000,2295.1348386540835],[1726617600000,2341.0924132597524],[1726704000000,2361.960992100706],[1726790400000,2465.2347833172967],[1726876800000,2557.6218555442515],[1726963200000,2614.599604753501],[1727049600000,2582.847683741262],[1727136000000,2647.9931622033255],[1727222400000,2653.844345308008],[1727308800000,2578.566359563475],[1727395200000,2630.94983700893],[1727481600000,2698.1928212346033],[1727568000000,2680.218702062471],[1727654400000,2659.611211611689],[1727740800000,2597.3411516834435],[1727827200000,2451.63152

In [5]:
eth_prices = pd.DataFrame(response.json()['prices'], columns=['timestamp', 'price'])
eth_maket_cap = pd.DataFrame(response.json()['market_caps'], columns=['timestamp', 'market_caps'])
eth_total_volume = pd.DataFrame(response.json()['total_volumes'], columns=['timestamp', 'total_volumes'])

In [6]:
eth_price = eth_prices.merge(eth_maket_cap, on='timestamp', how='left')
eth_price = eth_price.merge(eth_total_volume, on='timestamp', how='left')

In [9]:
eth_price

,timestamp,price,market_caps,total_volumes
0,2024-09-04 00:00:00,2437.147156,2.938183e+11,9.565142e+09
1,2024-09-05 00:00:00,2448.307858,2.943294e+11,1.663970e+10
2,2024-09-06 00:00:00,2367.770051,2.849891e+11,1.106019e+10
3,2024-09-07 00:00:00,2223.780428,2.675745e+11,2.608793e+10
4,2024-09-08 00:00:00,2272.697285,2.733503e+11,1.040519e+10
...,...,...,...,...
360,2025-08-30 00:00:00,4364.358521,5.268309e+11,3.800684e+10
361,2025-08-31 00:00:00,4373.360358,5.275395e+11,1.919918e+10
362,2025-09-01 00:00:00,4388.931465,5.297401e+11,2.226980e+10
363,2025-09-02 00:00:00,4303.202223,5.195348e+11,3.172520e+10


In [8]:
eth_price['timestamp'] = pd.to_datetime(eth_price['timestamp'], unit='ms')

In [10]:
eth_price.to_csv('../data/eth_price_market_cap_volume.csv', index=False)

## Curve metrics 

In [18]:
CHAINS = [
    'ARENAZ','AUTOMATA','BASE','BOB','CYBER','FRAX','HAM','INK','KROMA',
    'LISK','LYRA','METAL','MINT','MODE','OPTIMISM','ORDERLY','POLYNOMIAL',
    'RACE','REDSTONE','SHAPE','SONEIUM','SWAN','SWELL','UNICHAIN',
    'WORLDCHAIN','XTERIO','ZORA'
]
SOURCE_NAMES  = ['Github', 'Optimism', 'Base',    'Mode', 'Worldchain', 'Soneium', 'Ink',     'Unichain', 'Zora', 'Others']
TVL_METRIC = 'DefiLlama TVL (USD)'
FEES_METRIC = 'Fees Paid (ETH)'
TX_METRIC = 'Transaction Count'

START_DATE = '2022-01-01'
END_DATE   = '2025-07-31'
OSO_PROJECT_NAMES = list(['curve'])
ONCHAIN_METRICS = {
    'defillama_tvl_daily': 'DefiLlama TVL (USD)',
    'gas_fees_daily': 'Fees Paid (ETH)',
    'transactions_daily': 'Transaction Count',
#    'contract_invocations_daily': 'Contract Invocations Count',
}
# CODE_METRICS = {
#     'GITHUB_active_developers_monthly': 'Monthly Active Developers',
#     'GITHUB_full_time_developers_monthly': 'Monthly Full-Time Developers',
# }
ONCHAIN_METRIC_NAMES= [f"{c}_{m}" for c in CHAINS for m in ONCHAIN_METRICS.keys()]
# CODE_METRIC_NAMES = list(CODE_METRICS.keys())
# CODE_METRIC_LABELS = list(CODE_METRICS.values())

METRICS = ONCHAIN_METRIC_NAMES


df_metrics_raw = client.to_pandas(f"""

    SELECT DISTINCT
        sample_date,
        project_name,
        metric_name,
        amount
    FROM timeseries_metrics_by_project_v0
    JOIN metrics_v0 USING (metric_id)
    JOIN projects_v1 USING (project_id)
    WHERE
        project_name IN ({stringify(OSO_PROJECT_NAMES)})
        AND metric_name IN ({stringify(METRICS)})
        AND sample_date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
        AND project_source = 'OSS_DIRECTORY'

    ORDER BY sample_date, project_name, metric_name

""")

df_metrics_raw['chain'] = df_metrics_raw['metric_name'].apply(lambda x: x.split('_')[0])
df_metrics_raw['chain'] = df_metrics_raw['chain'].apply(lambda x: x.title() if x.title() in SOURCE_NAMES else 'Others')

df_metrics_raw['metric'] = df_metrics_raw['metric_name'].apply(lambda x: '_'.join(x.split('_')[1:]))
# df_metrics_raw['metric'] = df_metrics_raw['metric'].apply(lambda x: ONCHAIN_METRICS.get(x, CODE_METRICS.get(f'GITHUB_{x}')))
df_metrics_raw['sample_date'] = pd.to_datetime(df_metrics_raw['sample_date'])
df_metrics_raw.drop(columns=['metric_name'], inplace=True)

df_metrics_raw = df_metrics_raw.groupby(['sample_date', 'project_name', 'chain', 'metric']).sum().reset_index()
df_metrics_raw.to_parquet('../data/curve_metrics.parquet')

## Balancer metrics

In [3]:
CHAINS = [
    'ARENAZ','AUTOMATA','BASE','BOB','CYBER','FRAX','HAM','INK','KROMA',
    'LISK','LYRA','METAL','MINT','MODE','OPTIMISM','ORDERLY','POLYNOMIAL',
    'RACE','REDSTONE','SHAPE','SONEIUM','SWAN','SWELL','UNICHAIN',
    'WORLDCHAIN','XTERIO','ZORA'
]
SOURCE_NAMES  = ['Github', 'Optimism', 'Base',    'Mode', 'Worldchain', 'Soneium', 'Ink',     'Unichain', 'Zora', 'Others']
TVL_METRIC = 'DefiLlama TVL (USD)'
FEES_METRIC = 'Fees Paid (ETH)'
TX_METRIC = 'Transaction Count'

START_DATE = '2022-01-01'
END_DATE   = '2025-07-31'
OSO_PROJECT_NAMES = list(['balancer'])
ONCHAIN_METRICS = {
    'defillama_tvl_daily': 'DefiLlama TVL (USD)',
    'gas_fees_daily': 'Fees Paid (ETH)',
    'transactions_daily': 'Transaction Count',
#    'contract_invocations_daily': 'Contract Invocations Count',
}
# CODE_METRICS = {
#     'GITHUB_active_developers_monthly': 'Monthly Active Developers',
#     'GITHUB_full_time_developers_monthly': 'Monthly Full-Time Developers',
# }
ONCHAIN_METRIC_NAMES= [f"{c}_{m}" for c in CHAINS for m in ONCHAIN_METRICS.keys()]
# CODE_METRIC_NAMES = list(CODE_METRICS.keys())
# CODE_METRIC_LABELS = list(CODE_METRICS.values())

METRICS = ONCHAIN_METRIC_NAMES


df_metrics_raw = client.to_pandas(f"""

    SELECT DISTINCT
        sample_date,
        project_name,
        metric_name,
        amount
    FROM timeseries_metrics_by_project_v0
    JOIN metrics_v0 USING (metric_id)
    JOIN projects_v1 USING (project_id)
    WHERE
        project_name IN ({stringify(OSO_PROJECT_NAMES)})
        AND metric_name IN ({stringify(METRICS)})
        AND sample_date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
        AND project_source = 'OSS_DIRECTORY'

    ORDER BY sample_date, project_name, metric_name

""")

df_metrics_raw['chain'] = df_metrics_raw['metric_name'].apply(lambda x: x.split('_')[0])
df_metrics_raw['chain'] = df_metrics_raw['chain'].apply(lambda x: x.title() if x.title() in SOURCE_NAMES else 'Others')

df_metrics_raw['metric'] = df_metrics_raw['metric_name'].apply(lambda x: '_'.join(x.split('_')[1:]))
# df_metrics_raw['metric'] = df_metrics_raw['metric'].apply(lambda x: ONCHAIN_METRICS.get(x, CODE_METRICS.get(f'GITHUB_{x}')))
df_metrics_raw['sample_date'] = pd.to_datetime(df_metrics_raw['sample_date'])
df_metrics_raw.drop(columns=['metric_name'], inplace=True)

df_metrics_raw = df_metrics_raw.groupby(['sample_date', 'project_name', 'chain', 'metric']).sum().reset_index()
df_metrics_raw.to_parquet('../data/balancer_metrics.parquet')

In [4]:
df_metrics_raw

,sample_date,project_name,chain,metric,amount
0,2022-04-23,balancer,Optimism,gas_fees_daily,0.000005
1,2022-04-23,balancer,Optimism,transactions_daily,2.0
2,2022-05-14,balancer,Optimism,gas_fees_daily,0.001551
3,2022-05-14,balancer,Optimism,transactions_daily,1.0
4,2022-05-19,balancer,Optimism,gas_fees_daily,0.000004
...,...,...,...,...,...
7016,2025-07-31,balancer,Optimism,gas_fees_daily,0.00025
7017,2025-07-31,balancer,Optimism,transactions_daily,146.0
7018,2025-07-31,balancer,Others,defillama_tvl_daily,698657.0
7019,2025-07-31,balancer,Others,gas_fees_daily,0.000001


In [ ]:
## internal metrics by chain
CHAIN_METRICS = {
    'DEFILLAMA_TVL': TVL_METRIC,
    'TXCOUNT': TX_METRIC,
    'FEES_PAID_ETH': FEES_METRIC,
    # 'STABLES_MCAP': 'Stablecoin Market Cap (USD)',
    # 'MARKET_CAP_USD': 'Market Cap (USD)',
    # 'MARKET_CAP_ETH': 'Market Cap (ETH)',
    # 'ACTIVE_DEPLOYERS': 'Active Deployers',
    # 'CONTRACTS_DEPLOYED': 'Contracts Deployed',
}
 
df_chain_metrics_raw = client.to_pandas(f"""
SELECT
    sample_date,
    source,
    chain,
    metric_name,
    amount,
    SUM(amount / 30) OVER (
        PARTITION BY source, chain, metric_name
        ORDER BY sample_date
        RANGE BETWEEN INTERVAL '29' DAY PRECEDING AND CURRENT ROW
    ) AS rolling_30d_amount
FROM int_chain_metrics
WHERE
chain IN ({stringify(CHAINS)})
AND metric_name IN ({stringify(CHAIN_METRICS.keys())})
AND sample_date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
""")

df_chain_metrics_raw['sample_date'] = pd.to_datetime(df_chain_metrics_raw['sample_date'])
df_chain_metrics_raw['chain'] = df_chain_metrics_raw['chain'].apply(lambda x: x.title() if x.title() in SOURCE_NAMES else 'Others')
df_chain_metrics_raw['metric'] = df_chain_metrics_raw['metric_name'].map(CHAIN_METRICS)
df_chain_metrics_raw = df_chain_metrics_raw.groupby(['sample_date', 'source', 'chain', 'metric'])[['amount', 'rolling_30d_amount']].sum().reset_index()

df_chain_metrics_raw.to_parquet('../data/chain_metrics.parquet')


c:\Users\vieir\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\vieir\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


,sample_date,source,chain,metric,amount,rolling_30d_amount
0,2022-01-01,DEFILLAMA,Optimism,DefiLlama TVL (USD),691405700.0,46093713.333333
1,2022-01-01,GROWTHEPIE,Optimism,Fees Paid (ETH),16.619587,0.553986
2,2022-01-01,GROWTHEPIE,Optimism,Transaction Count,27621.0,920.7
3,2022-01-02,DEFILLAMA,Optimism,DefiLlama TVL (USD),696860850.0,92551103.333333
4,2022-01-02,GROWTHEPIE,Optimism,Fees Paid (ETH),18.365874,1.166182
...,...,...,...,...,...,...
15020,2025-07-31,GROWTHEPIE,Unichain,Transaction Count,1091127.0,1354695.1
15021,2025-07-31,GROWTHEPIE,Worldchain,Fees Paid (ETH),0.534451,3.22457
15022,2025-07-31,GROWTHEPIE,Worldchain,Transaction Count,1581090.0,1557927.966667
15023,2025-07-31,GROWTHEPIE,Zora,Fees Paid (ETH),0.007564,0.012377


In [8]:
df_chain_metrics_raw.to_parquet('../data/chain_metrics.parquet')

In [1]:
## Ethereum count of transactions 

In [2]:
import requests
import pandas as pd

url = 'https://api.growthepie.xyz/v1/export/txcount.json'
response = requests.get(url)
df = pd.DataFrame(response.json())

In [5]:
df[df.origin_key == 'ethereum'].to_csv('../data/eth_txcount.csv', index=False)